---
title: "Lab 6: Variable Selection and Regularization"
author: "James Compagno"
format: 
  html:
    code-fold: true
    code-line-numbers: true
    code-tools: true
    self-contained: true
execute:
  message: false
  echo: false
  eval: false
---

In [12]:
import pandas as pd
import numpy as np
import plotnine as p9
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

# Dataset: Baseball Players

In this lab, we will use predictive modeling to design a model that predicts a baseball player's salary in a given year.

This dataset was originally taken from the StatLib library which is maintained at Carnegie Mellon University. This is part of the data that was used in the 1988 ASA Graphics Section Poster Session. The salary data were originally from Sports Illustrated, April 20, 1987. The 1986 and career statistics were obtained from The 1987 Baseball Encyclopedia Update published by Collier Books, Macmillan Publishing Company, New York.

**Format:** A data frame with 322 observations of major league players on the following 20 variables.

`AtBat` Number of times at bat in 1986 

`Hits` Number of hits in 1986 

`HmRun` Number of home runs in 1986

`Runs` Number of runs in 1986 

`RBI` Number of runs batted in in 1986 

`Walks` Number of walks in 1986 

`Years` Number of years in the major leagues 

`CAtBat` Number of times at bat during his career 

`CHits` Number of hits during his career 

`CHmRun` Number of home runs during his career 

`CRuns` Number of runs during his career 

`CRBI` Number of runs batted in during his career 

`CWalks` Number of walks during his career 

`League` A factor with levels A and N indicating player's league at the end of 1986 

`Division` A factor with levels E and W indicating player's division at the end of 1986 

`PutOuts` Number of put outs in 1986 

`Assists` Number of assists in 1986 

`Errors` Number of errors in 1986 

`Salary` 1987 annual salary on opening day in thousands of dollars 

`NewLeague` A factor with levels A and N indicating player's league at the beginning of 1987

You can download the dataset from [here](https://www.dropbox.com/s/boshaqfgdjiaxh4/Hitters.csv?dl=1).

A couple notes about this lab:

1.  Although it isn't listed as a specific question, don't forget to clean your data at the beginning. How will you handle missing data? Are there any variables that need adjusting?

2.  There are a **lot** of variables in the dataset! You may want to use the `remainder = "passthrough"` trick in your column transformers, rather than typing out a ton of gene names.

3.  Don't forget that in penalized regression, we **must** standardize our numeric variables.

4.  There is a lot of repetition in this lab. Think about ways to streamline your code - for example, you might consider writing simple functions to easily create pipelines.


In [13]:
# Read the data
hitters = pd.read_csv("Hitters.csv")

# Remove rows with no value for salary 
hitters = hitters.dropna(subset=["Salary"])
hitters = hitters.reset_index(drop=True)

# Assign values
X = hitters.drop(columns=["Salary"])
y = hitters["Salary"]

# Train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=67)

# Model Library 
model_library = {}
records = []

In [14]:
# Column Transformer 
ct = ColumnTransformer(
    [
        ("standardize", 
         StandardScaler(), 
         make_column_selector(dtype_include=np.number)),
        ("cat", 
         OneHotEncoder(drop="first", sparse_output=False), 
         make_column_selector(dtype_include=object))
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

# Part I: Different Model Specs

## A. Regression without regularization

1.  Create a pipeline that includes *all* the columns as predictors for `Salary`, and performs ordinary linear regression

2.  Fit this pipeline to the full dataset, and interpret a few of the most important coefficients.

3.  Use cross-validation to estimate the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [ ]:
# Model Name
model_name = "All_Features"
regression_type = "Linear"  

# Cross Validation Pipeline
pipe = Pipeline([
    ("preprocess", ct),
    ("linear_regression", LinearRegression())
])

# Add to Library
model_library[model_name] = pipe.fit(X, y)

# Metrics Calculation 
rmse = cross_val_score(pipe, X, y, cv=5, scoring='neg_root_mean_squared_error')
mse = cross_val_score(pipe, X, y, cv=5, scoring='neg_mean_squared_error')
r2 = cross_val_score(pipe, X, y, cv=5, scoring='r2')

# Metrics Storage 
records.append({
    "Model": model_name,
    "Regression Type": regression_type,
    "Best Alpha": "NA", 
    "Best L1 Ratio": "NA", 
    "Split": "CV-5",
    "RMSE Mean": -rmse.mean(),
    "MSE Mean": -mse.mean(),
    "R2 Mean": r2.mean()
})

# Display
cumulative_models = (pd.DataFrame(records))
cumulative_models

,Model,Regression Type,Best Alpha,Split,RMSE Mean,MSE Mean,R2 Mean
0,All_Features,Linear,NA,CV-5,342.139591,121136.310318,0.343495


In [16]:
# Get coefficients
fitted_pipe = model_library[model_name]
feature_names = fitted_pipe.named_steps['preprocess'].get_feature_names_out()
coefficients = fitted_pipe.named_steps['linear_regression'].coef_


coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
}).sort_values('Coefficient', key=abs, ascending=False)

# Top 10 coefficients
print(coef_df.head(10))  

       Feature  Coefficient
10       CRuns   480.747135
7       CAtBat  -391.038655
1         Hits   337.830479
0        AtBat  -291.094556
11        CRBI   260.689886
12      CWalks  -213.892259
5        Walks   135.073897
17  Division_W  -116.849246
8        CHits    86.687617
13     PutOuts    78.761296


Interpretation: The top Coefficients for salary: CRuns (Career Runs), CAtBat (Career At-Bats), Hits (1986 Hits), etc. are what you would expec: more time playing = more mony. However some coefficients are negative, likely due to multicollinearity as a person who has more bats will have more hits who will have more runs. 

## B. Ridge regression

1.  Create a pipeline that includes *all* the columns as predictors for `Salary`, and performs ordinary ridge regression

2.  Use cross-validation to **tune** the $\lambda$ hyperparameter.

3.  Fit the pipeline with your chosen $\lambda$ to the full dataset, and interpret a few of the most important coefficients.

4.  Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [ ]:
# Model Name
model_name = "Ridge_All_Tuned"
regression_type = "Ridge"  

# Pipeline with Ridge
pipe = Pipeline([
    ("preprocess", ct),
    ("ridge", Ridge())  
])

# GridSearchCV 
param_grid = {'ridge__alpha': np.logspace(-2, 3, 50)}
gscv = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_squared_error')
gscv.fit(X, y)

# Add best model to Library
model_library[model_name] = gscv.best_estimator_

# Get best alpha
best_alpha = gscv.best_params_['ridge__alpha']

# Metrics Calculation using best model
rmse = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='neg_root_mean_squared_error')
mse = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')
r2 = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='r2')

# Metrics Storage 
records.append({
    "Model": model_name,
    "Regression Type": regression_type,
    "Best Alpha": best_alpha, 
    "Best L1 Ratio": "NA", 
    "Split": "CV-5",
    "RMSE Mean": -rmse.mean(),
    "MSE Mean": -mse.mean(),
    "R2 Mean": r2.mean()
})

# Display
cumulative_models = pd.DataFrame(records)
cumulative_models

Best alpha: 3.5565


,Model,Regression Type,Best Alpha,Split,RMSE Mean,MSE Mean,R2 Mean
0,All_Features,Linear,NA,CV-5,342.139591,121136.310318,0.343495
1,Ridge_All_Tuned,Ridge,3.55648,CV-5,338.622793,118715.880634,0.363827


In [19]:
# Get coefficients
fitted_pipe = model_library[model_name]
feature_names = fitted_pipe.named_steps['preprocess'].get_feature_names_out()
coefficients = fitted_pipe.named_steps['ridge'].coef_


coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
}).sort_values('Coefficient', key=abs, ascending=False)

# Top 10 coefficients
print(coef_df.head(10)) 

       Feature  Coefficient
1         Hits   230.162932
0        AtBat  -215.899174
10       CRuns   199.138372
12      CWalks  -144.147121
8        CHits   119.239855
17  Division_W  -118.560116
11        CRBI   114.327122
5        Walks   106.094776
7       CAtBat   -93.427352
13     PutOuts    77.671049


Interpretation: The coefficients for ridge regression were very similar to normal linear but in a different order and the coefficient number are smaller.. There is still an issue with multicollinearity 

## C. Lasso Regression

1.  Create a pipeline that includes *all* the columns as predictors for `Salary`, and performs ordinary ridge regression

2.  Use cross-validation to **tune** the $\lambda$ hyperparameter.

3.  Fit the pipeline with your chosen $\lambda$ to the full dataset, and interpret a few of the most important coefficients.

4.  Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [ ]:
# Model Name
model_name = "Lasso_All_Tuned"
regression_type = "Lasso"  

# Pipeline
pipe = Pipeline([
    ("preprocess", ct),
    ("lasso", Lasso()) 
])

# GridSearchCV 
param_grid = {'lasso__alpha': np.logspace(-2, 3, 50)} 
gscv = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_squared_error')
gscv.fit(X, y)

# Add best model to Library
model_library[model_name] = gscv.best_estimator_

# Get best alpha
best_alpha = gscv.best_params_['lasso__alpha']

# Metrics Calculation using best model
rmse = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='neg_root_mean_squared_error')
mse = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')
r2 = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='r2')

# Metrics Storage 
records.append({
    "Model": model_name,
    "Regression Type": regression_type,
    "Best Alpha": best_alpha, 
    "Best L1 Ratio": "NA", 
    "Split": "CV-5",
    "RMSE Mean": -rmse.mean(),
    "MSE Mean": -mse.mean(),
    "R2 Mean": r2.mean()
})

# Display
cumulative_models = pd.DataFrame(records)
cumulative_models

/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.591e+06, tolerance: 4.708e+03
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.056e+06, tolerance: 3.606e+03
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.589e+06, tolerance: 4.137e+03
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coor

,Model,Regression Type,Best Alpha,Split,RMSE Mean,MSE Mean,R2 Mean
0,All_Features,Linear,NA,CV-5,342.139591,121136.310318,0.343495
1,Ridge_All_Tuned,Ridge,3.55648,CV-5,338.622793,118715.880634,0.363827
2,Lasso_All_Tuned,Lasso,2.222996,CV-5,339.164091,119077.752512,0.364623


In [24]:
# Get coefficients
fitted_pipe = model_library[model_name]
feature_names = fitted_pipe.named_steps['preprocess'].get_feature_names_out()
coefficients = fitted_pipe.named_steps['lasso'].coef_


coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
}).sort_values('Coefficient', key=abs, ascending=False)

# Top 10 coefficients
print(coef_df.head(10)) 

       Feature  Coefficient
1         Hits   269.912291
0        AtBat  -249.356171
10       CRuns   233.402901
12      CWalks  -153.722044
11        CRBI   122.593538
17  Division_W  -114.745451
5        Walks   107.991980
13     PutOuts    77.414749
6        Years   -48.707831
9       CHmRun    47.327574


## D. Elastic Net

1.  Create a pipeline that includes *all* the columns as predictors for `Salary`, and performs ordinary ridge regression

2.  Use cross-validation to **tune** the $\lambda$ and $\alpha$ hyperparameters.

3.  Fit the pipeline with your chosen hyperparameters to the full dataset, and interpret a few of the most important coefficients.

4.  Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [ ]:
# Model Name
model_name = "ElasticNet_All_Tuned"  
regression_type = "ElasticNet" 

# Pipeline with ElasticNet
pipe = Pipeline([
    ("preprocess", ct),
    ("elasticnet", ElasticNet()) 
])

# GridSearchCV 
param_grid = {
    'elasticnet__alpha': np.logspace(-2, 3, 50),
    'elasticnet__l1_ratio': [0, 0.25, 0.5, 0.75, 1]
} 
gscv = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_squared_error')
gscv.fit(X, y)

# Add best model to Library
model_library[model_name] = gscv.best_estimator_

# Get best Alpha and L1
best_alpha = gscv.best_params_['elasticnet__alpha'] 
best_l1_ratio = gscv.best_params_['elasticnet__l1_ratio'] 

print(f"Best alpha: {best_alpha:.4f}")
print(f"Best l1_ratio: {best_l1_ratio}")

# Metrics Calculation using best model
rmse = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='neg_root_mean_squared_error')
mse = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')
r2 = cross_val_score(gscv.best_estimator_, X, y, cv=5, scoring='r2')

# Metrics Storage 
records.append({
    "Model": model_name,
    "Regression Type": regression_type,
    "Best Alpha": best_alpha,
    "Best L1 Ratio": best_l1_ratio, 
    "Split": "CV-5",
    "RMSE Mean": -rmse.mean(),
    "MSE Mean": -mse.mean(),
    "R2 Mean": r2.mean()
})

# Display
cumulative_models = pd.DataFrame(records)
cumulative_models

/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.085e+07, tolerance: 4.708e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.477e+06, tolerance: 3.606e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
/opt/anaco

Best alpha: 0.0160
Best l1_ratio: 0


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.347e+07, tolerance: 4.708e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.798e+07, tolerance: 3.606e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
/opt/anaco

,Model,Regression Type,Best Alpha,Split,RMSE Mean,MSE Mean,R2 Mean,Best L1 Ratio
0,All_Features,Linear,NA,CV-5,342.139591,121136.310318,0.343495,NaN
1,Ridge_All_Tuned,Ridge,3.55648,CV-5,338.622793,118715.880634,0.363827,NaN
2,Lasso_All_Tuned,Lasso,2.222996,CV-5,339.164091,119077.752512,0.364623,NaN
3,ElasticNet_All_Tuned,ElasticNet,0.015999,CV-5,338.639729,118712.292070,0.363464,0.0


In [29]:
def show_top_coefficients(model_name, step_name='linear_regression', n=10):
    """Show top n coefficients by absolute value"""
    pipe = model_library[model_name]
    features = pipe.named_steps['preprocess'].get_feature_names_out()
    coefs = pipe.named_steps[step_name].coef_
    
    return pd.DataFrame({
        'Feature': features,
        'Coefficient': coefs
    }).sort_values('Coefficient', key=abs, ascending=False).head(n)

In [30]:
show_top_coefficients(ElasticNet_All_Tuned, elasticnet, n=10)

NameError: name 'ElasticNet_All_Tuned' is not defined

In [27]:
# Get coefficients
fitted_pipe = model_library[model_name]
feature_names = fitted_pipe.named_steps['preprocess'].get_feature_names_out()
coefficients = fitted_pipe.named_steps['elasticnet'].coef_


coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
}).sort_values('Coefficient', key=abs, ascending=False)

# Top 10 coefficients
print(coef_df.head(10)) 

       Feature  Coefficient
1         Hits   218.136088
0        AtBat  -204.423514
10       CRuns   184.208473
12      CWalks  -136.809717
17  Division_W  -117.995800
8        CHits   116.499537
11        CRBI   110.189848
5        Walks   102.631546
7       CAtBat   -78.621388
13     PutOuts    77.421908


# Part II. Variable Selection

Based on the above results, decide on:

-   Which *numeric* variable is most important: 

        Hits

-   Which *five* numeric variables are most important:

        1. CRuns
        2. Hits
        3. AtBat
        4. CRBI
        5. CWalks

-   Which *categorical* variable is most important: 
        
        Division_W

For **each** of the four model specifications, compare the following possible feature sets:

1.  Using only the one best numeric variable.

2.  Using only the five best variables.

3.  Using the five best numeric variables *and* their interactions with the one best categorical variable.

Report which combination of features and model performed best, based on the validation metric of MSE.

(Note: $\lambda$ and $\alpha$ must be re-tuned for each feature set.)

In [ ]:
def run_linear_regression(model_name, features=None):
    """
    model_name - will be stored in model_library
    features - list or None
    Returns: DataFrame 
    """
    regression_type = "Linear"
    
    # Select features
    if features is not None:
        X_subset = X[features]
    else:
        X_subset = X
    
    # Cross Validation Pipeline
    pipe = Pipeline([
        ("preprocess", ct),
        ("linear_regression", LinearRegression())
    ])
    
    # Fit and add to Library
    pipe.fit(X_subset, y)
    model_library[model_name] = pipe
    
    # Get top 10 coefficients
    feature_names = pipe.named_steps['preprocess'].get_feature_names_out()
    coefficients = pipe.named_steps['linear_regression'].coef_
    
    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients
    }).sort_values('Coefficient', key=abs, ascending=False).head(10)
    
    # Store coefficients in model library
    model_library[f"{model_name}_coefficients"] = coef_df
    
    # Metrics Calculation 
    rmse = cross_val_score(pipe, X_subset, y, cv=5, scoring='neg_root_mean_squared_error')
    mse = cross_val_score(pipe, X_subset, y, cv=5, scoring='neg_mean_squared_error')
    r2 = cross_val_score(pipe, X_subset, y, cv=5, scoring='r2')
    
    # Metrics Storage 
    records.append({
        "Model": model_name,
        "Regression Type": regression_type,
        "Best Alpha": "NA", 
        "Best L1 Ratio": "NA",
        "Variables Used": len(X_subset.columns) if features else "All",
        "Top 10 Variables": 
        "Top 10 Coefficients":
        "Split": "CV-5",
        "RMSE Mean": -rmse.mean(),
        "MSE Mean": -mse.mean(),
        "R2 Mean": r2.mean()
    })
    
    # Display
    cumulative_models = pd.DataFrame(records)
    return cumulative_models

In [40]:
def run_linear_regression(model_name, features=None):
    """
    model_name - will be stored in model_library
    features - list or None
    Returns: DataFrame 
    """
    regression_type = "Linear"
    
    # Select features
    if features is not None:
        X_subset = X[features]
    else:
        X_subset = X
    
    # Cross Validation Pipeline
    pipe = Pipeline([
        ("preprocess", ct),
        ("linear_regression", LinearRegression())
    ])
    
    # Fit and add to Library
    pipe.fit(X_subset, y)
    model_library[model_name] = pipe
    
    # Get top 10 coefficients
    feature_names = pipe.named_steps['preprocess'].get_feature_names_out()
    coefficients = pipe.named_steps['linear_regression'].coef_
    
    coef_df = pd.DataFrame({
        'Variable': feature_names,
        'Coefficient': coefficients
    }).sort_values('Coefficient', key=abs, ascending=False).head(10)
    
    # Store coefficients in model library
    model_library[f"{model_name}_coefficients"] = coef_df
    
    # Metrics Calculation 
    rmse = cross_val_score(pipe, X_subset, y, cv=5, scoring='neg_root_mean_squared_error')
    mse = cross_val_score(pipe, X_subset, y, cv=5, scoring='neg_mean_squared_error')
    r2 = cross_val_score(pipe, X_subset, y, cv=5, scoring='r2')
    
    # Metrics Storage 
    records.append({
        "Model": model_name,
        "Regression Type": regression_type,
        "Best Alpha": "NA", 
        "Best L1 Ratio": "NA",
        "Variables Used": len(X_subset.columns) if features else "All",
        "Top 10 Variables": ", ".join(coef_df['Variable'].head(10).tolist()),
        "Top 10 Coefficients": ", ".join([f"{c:.2f}" for c in coef_df['Coefficient'].head(10).tolist()]),
        "Split": "CV-5",
        "RMSE Mean": -rmse.mean(),
        "MSE Mean": -mse.mean(),
        "R2 Mean": r2.mean()
    })
    
    # Display
    cumulative_models = pd.DataFrame(records)
    return cumulative_models

In [41]:
run_linear_regression("LinearFuntTest2", None)

,Model,Regression Type,Best Alpha,Split,RMSE Mean,MSE Mean,R2 Mean,Best L1 Ratio,Features Used,Variables Used,Top 10 Variables,Top 10 Coefficients
0,All_Features,Linear,NA,CV-5,342.139591,121136.310318,0.343495,NaN,NaN,NaN,NaN,NaN
1,Ridge_All_Tuned,Ridge,3.55648,CV-5,338.622793,118715.880634,0.363827,NaN,NaN,NaN,NaN,NaN
2,Lasso_All_Tuned,Lasso,2.222996,CV-5,339.164091,119077.752512,0.364623,NaN,NaN,NaN,NaN,NaN
3,ElasticNet_All_Tuned,ElasticNet,0.015999,CV-5,338.639729,118712.292070,0.363464,0,NaN,NaN,NaN,NaN
4,LinearFuntTest,Linear,NA,CV-5,342.139591,121136.310318,0.343495,NA,All,NaN,NaN,NaN
5,LinearFuntTest,Linear,NA,CV-5,342.139591,121136.310318,0.343495,NA,All,NaN,NaN,NaN
6,LinearFuntTest,Linear,NA,CV-5,342.139591,121136.310318,0.343495,NA,All,NaN,NaN,NaN
7,LinearFuntTest2,Linear,NA,CV-5,342.139591,121136.310318,0.343495,NA,NaN,All,"CRuns, CAtBat, Hits, AtBat, CRBI, CWalks, Walk...","480.75, -391.04, 337.83, -291.09, 260.69, -213..."


# Part III. Discussion

## A. Ridge

Compare your Ridge models with your ordinary regression models. How did your coefficients compare? Why does this make sense?

## B. LASSO

Compare your LASSO model in I with your three LASSO models in II. Did you get the same $\lambda$ results? Why does this make sense? Did you get the same MSEs? Why does this make sense?

## C. Elastic Net

Compare your MSEs for the Elastic Net models with those for the Ridge and LASSO models. Why does it make sense that Elastic Net always "wins"?

# Part IV: Final Model

Fit your final best pipeline on the full dataset, and summarize your results in a few short sentences and a plot.